In [1]:
import math
import random
import time
import numpy as np

In [2]:
def american_to_percentage(odds):
    if odds >= 100:
        return 100 / (odds + 100)
    elif odds <= -100:
        return abs(odds) / (abs(odds) + 100)
    else:
        print("Invalid Number, cannot be between -100 and 100")
        return None
    
def percentage_to_american(probability):
    if probability == 0:
        return None
    elif probability > 0.5:
        return round((probability * 100) / (1 - probability) * -1)
    else:
        return round((100 / probability) - 100)
    
def calculate_winnings(odds, bet_amount):
    if odds > 0:
        winnings = (odds / 100) * bet_amount
    else:
        winnings = (100 / abs(odds)) * bet_amount
    
    return round(winnings, 2)

def parlay(odds_list):
    parlay_probability = 1
    for odds in odds_list:
        parlay_probability *= american_to_percentage(odds)
    return parlay_probability   
    

In [31]:
def simulate_betting(odds=-135, paid_odds=-110, goal=3000, initial_balance=100, bet_percentage=0.1, iterations=10_000, max_bets=1_000_000):
    win = 0
    lost = 0
    bets_to_goal = []  # List to store the number of bets to reach the goal for each successful session

    for _ in range(iterations):
        current_balance = initial_balance
        bets = 0

        while 0 < current_balance < goal and bets < max_bets:
            bets += 1
            bet_amount = current_balance * bet_percentage  # Bet a percentage of the current balance
            
            # Ensure the bet does not exceed the current balance
            if bet_amount > current_balance:
                bet_amount = current_balance

            # Deduct bet from current balance
            current_balance -= bet_amount

            # Check if the bet is won
            if random.random() < american_to_percentage(odds):
                # Winning outcome
                winnings = calculate_winnings(paid_odds, bet_amount)
                current_balance += winnings + bet_amount
            else:
                # Losing outcome; bet remains a percentage of the new balance next round
                pass

            # Break if goal is reached
            if current_balance >= goal:
                bets_to_goal.append(bets)  # Track number of bets for this successful session
                break

        # Track wins and losses
        if current_balance >= goal:
            win += 1
        else:
            lost += 1

    # Calculate average bets to reach the goal for successful sessions
    average_bets_to_goal = sum(bets_to_goal) / len(bets_to_goal) if bets_to_goal else 0

    # Output results
    print("WON:", win)
    print("LOST:", lost)
    print("Chances of Succeeding:", win / iterations)
    print("Average Bets to Reach Goal:", average_bets_to_goal)


In [50]:
simulate_betting(
    odds=-135, 
    paid_odds=-110, 
    goal=2000, 
    initial_balance=100, 
    bet_percentage=0.1,
    iterations=1000, 
    max_bets=1_000_000
)

WON: 999
LOST: 1
Chances of Succeeding: 0.999
Average Bets to Reach Goal: 613.1771771771772
